## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

                                                                                                  
 activation_9 (Activation)      (None, 32, 32, 64)   0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 16, 16, 64)   4160        ['activation_9[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 16, 16, 64)  256         ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_10 (Activation)     (None, 16, 16, 64)   0           ['batch_normalization_10[0][0]'] 
                                                                                                  
 conv2d_12

                                                                                                  
 conv2d_22 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_19[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 8, 8, 128)   512         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_20 (Activation)     (None, 8, 8, 128)    0           ['batch_normalization_20[0][0]'] 
                                                                                                  
 conv2d_24 (Conv2D)             (None, 8, 8, 256)    33024       ['add_5[0][0]']                  
                                                                                                  
 conv2d_23

In [4]:
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
    rotation_range=40, # 角度值，0~180，影象旋轉
    width_shift_range=0.2, # 水平平移，相對總寬度的比例
    height_shift_range=0.2, # 垂直平移，相對總高度的比例
    shear_range=0.2, # 隨機錯切換角度
    zoom_range=0.2, # 隨機縮放範圍
    horizontal_flip=True, # 一半影象水平翻轉
    fill_mode='nearest' # 填充新建立畫素的方法
)

In [5]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))

model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), 
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\jmk80\AppData\Local\Temp\ipykernel_27076\965074214.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),


Epoch 1/30
781/781 [==============================] - 659s 839ms/step - loss: 2.0911 - accuracy: 0.4122 - val_loss: 2.2044 - val_accuracy: 0.4126
Epoch 2/30
781/781 [==============================] - 662s 847ms/step - loss: 1.7001 - accuracy: 0.5088 - val_loss: 1.6566 - val_accuracy: 0.5108
Epoch 3/30
781/781 [==============================] - 609s 780ms/step - loss: 1.5200 - accuracy: 0.5583 - val_loss: 1.4503 - val_accuracy: 0.5897
Epoch 4/30
781/781 [==============================] - 596s 763ms/step - loss: 1.4080 - accuracy: 0.5948 - val_loss: 2.1024 - val_accuracy: 0.4756
Epoch 5/30
781/781 [==============================] - 599s 767ms/step - loss: 1.3237 - accuracy: 0.6224 - val_loss: 1.4351 - val_accuracy: 0.6054
Epoch 6/30
781/781 [==============================] - 599s 766ms/step - loss: 1.2600 - accuracy: 0.6440 - val_loss: 1.6780 - val_accuracy: 0.5402
Epoch 7/30
781/781 [==============================] - 599s 767ms/step - loss: 1.1970 - accuracy: 0.6653 - val_loss: 1.8141 -